In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import pickle
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
cd '/Users/snehavenkat/Desktop/BDS/ml-100k' 

/Users/snehavenkat/Desktop/BDS/ml-100k


In [3]:
# Reading ratings file
#ratings = pd.read_csv('ratings.csv', sep='|', encoding='latin-1', usecols=['userId', 'movieId', 'rating', 'timestamp'])
ratings = pd.read_csv("u.data",sep='\t',names=['user_id','movie_id','rating','timestamp'])
ratings = ratings[ratings.movie_id != 267]
ratings = ratings[ratings.movie_id != 1373]
# Reading users file
users = pd.read_csv('u.user', sep='|', encoding='latin-1', names=['user_id', 'age_desc', 'gender','occ_desc','zipcode'])
# Reading movies file
movies = pd.read_csv('movies.csv', sep='$', encoding='latin-1', names=['movie_id', 'title', 'genres'])

In [4]:
# Break up the big genre string into a string array
movies['genres'] = movies['genres'].str.split('|')
#print(movies['genres'])
# Convert genres to string value
movies['genres'] = movies['genres'].fillna("").astype('str')

In [5]:
#TFIDF calculation
train_time = time.time()
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres'])
# pp = pickle.dumps(tfidf_matrix) #Saving the tf-idf matric
# print(sys.getsizeof(pp))
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
# pp = pickle.dumps(cosine_sim) #Saving the linear kernel
# print(sys.getsizeof(pp))
print("--- Train time is %s seconds ---" % (time.time() - train_time))

--- Train time is 0.0516269207001 seconds ---


In [6]:
 # Build a 1-dimensional array with movie titles
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])

# Function that get movie recommendations based on the cosine similarity score of movie genres
def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [7]:
test_time = time.time()
user_vector = ratings[ratings.user_id==120] #user id 120
max_rating = max(user_vector.rating)
for index,row in user_vector.iterrows():
    if row['rating']==max_rating.astype(int):
        get_movie = movies[movies.movie_id==row[1]]
        print(genre_recommendations(str(get_movie.title.iloc[0]))).head(5)#Print top 5 movies
    print" "
print("--- Test time is %s seconds ---" % (time.time() - test_time))

 
 
24                                 Birdcage, The (1996)
25                        Brothers McMullen, The (1995)
39    To Wong Foo, Thanks for Everything! Julie Newm...
40                                 Billy Madison (1995)
41                                        Clerks (1994)
Name: title, dtype: object
 
 
 
 
 
 
284     English Patient, The (1996)
481               Casablanca (1942)
547                  Rob Roy (1995)
1482     Colonel Chabert, Le (1994)
1122     Farewell to Arms, A (1932)
Name: title, dtype: object
 
 
 
 
 
257                                        Contact (1997)
427                 Day the Earth Stood Still, The (1951)
1004    Until the End of the World (Bis ans Ende der W...
268                                        Gattaca (1997)
238                                       Sneakers (1992)
Name: title, dtype: object
 
 
 
 
 
 
 
 
 
 
187    Full Metal Jacket (1987)
324            G.I. Jane (1997)
513            Boot, Das (1981)
649                Glory (1